<a href="https://colab.research.google.com/github/priyanshgupta1998/Machine_learning/blob/master/Adult_Income_DesisonTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/Index

In [0]:
find = open('Index' , 'r+')
print(find.read())

In [68]:
f = open('adult.names' , 'r+')
lines = f.readlines()
# lines[92:]
column = []
for i in lines[96:]:
  column.append(i.split(':')[0])
  
column+=['cross_validator']
print(len(column))
print(column)

15
['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'cross_validator']


In [72]:
import pandas as pd
# df = pd.DataFrame(columns = column)
df1 = pd.read_csv('adult.data' , names = column)
# df1.to_csv('Train.csv' , index=False)
# get = df1.columns
# df1.columns = column
print(df1.shape)
df1.head()

(32561, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,cross_validator
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [79]:
import pandas as pd
df2 = pd.read_csv('adult.test', names = column )
# df2.columns = column
df2.drop(0 , axis=0 , inplace =True)
print(df2.shape)
df2.head()

(16281, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,cross_validator
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
5,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.


In [0]:
train = df1.copy()
test = df2.copy()

#work on training part 


In [140]:
import numpy as np
df =train.copy()

# filling missing values
col_names = df.columns
for c in col_names:
    df[c] = df[c].replace(" ?", np.NaN)

print(df.isnull().sum())

age                   0
workclass          1836
fnlwgt                0
education             0
education-num         0
marital-status        0
occupation         1843
relationship          0
race                  0
sex                   0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      583
cross_validator       0
dtype: int64


In [141]:
df = df.apply(lambda x:x.fillna(x.value_counts().index[0]))
df.info()   # There is no any null value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age                32561 non-null int64
workclass          32561 non-null object
fnlwgt             32561 non-null int64
education          32561 non-null object
education-num      32561 non-null int64
marital-status     32561 non-null object
occupation         32561 non-null object
relationship       32561 non-null object
race               32561 non-null object
sex                32561 non-null object
capital-gain       32561 non-null int64
capital-loss       32561 non-null int64
hours-per-week     32561 non-null int64
native-country     32561 non-null object
cross_validator    32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [142]:
#discretisation
df['marital-status'].value_counts()

 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital-status, dtype: int64

In [123]:
old_ = ['Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent','Never-married','Separated','Widowed']
new_ = ['divorced','married','married','married','not-married','not-married','not-married']
dicc = {}
for i in range(len(old_)):
  dicc[old_[i]] = new_[i]
  
print(dicc)

{'Divorced': 'divorced', 'Married-AF-spouse': 'married', 'Married-civ-spouse': 'married', 'Married-spouse-absent': 'married', 'Never-married': 'not-married', 'Separated': 'not-married', 'Widowed': 'not-married'}


In [0]:
#discretisation
df['marital-status'] = df['marital-status'].apply(lambda x: dicc[x.strip()])

In [133]:
df['marital-status'].value_counts()

married        15417
not-married    12701
divorced        4443
Name: marital-status, dtype: int64

In [144]:
df.drop(['fnlwgt'  , 'education-num'], axis = 1 , inplace=True)
df.head(3)

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,cross_validator
0,39,State-gov,Bachelors,not-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,married,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [145]:
df['cross_validator'].value_counts()

 <=50K    24720
 >50K      7841
Name: cross_validator, dtype: int64

In [0]:
df['income'] = df['cross_validator']
df.drop('cross_validator' , axis = 1 ,inplace=True)

In [147]:
from sklearn import preprocessing

#label Encoder
category_col =['workclass', 'race', 'education','marital-status', 'occupation','relationship', 'sex', 'native-country', 'income'] 
labelEncoder = preprocessing.LabelEncoder()

# creating a map of all the numerical values of each categorical labels.
mapping_dict={}
for col in category_col:
    df[col] = labelEncoder.fit_transform(df[col])
    le_name_mapping = dict(zip(labelEncoder.classes_, labelEncoder.transform(labelEncoder.classes_)))
    mapping_dict[col]=le_name_mapping
print(mapping_dict)


{'workclass': {' Federal-gov': 0, ' Local-gov': 1, ' Never-worked': 2, ' Private': 3, ' Self-emp-inc': 4, ' Self-emp-not-inc': 5, ' State-gov': 6, ' Without-pay': 7}, 'race': {' Amer-Indian-Eskimo': 0, ' Asian-Pac-Islander': 1, ' Black': 2, ' Other': 3, ' White': 4}, 'education': {' 10th': 0, ' 11th': 1, ' 12th': 2, ' 1st-4th': 3, ' 5th-6th': 4, ' 7th-8th': 5, ' 9th': 6, ' Assoc-acdm': 7, ' Assoc-voc': 8, ' Bachelors': 9, ' Doctorate': 10, ' HS-grad': 11, ' Masters': 12, ' Preschool': 13, ' Prof-school': 14, ' Some-college': 15}, 'marital-status': {'divorced': 0, 'married': 1, 'not-married': 2}, 'occupation': {' Adm-clerical': 0, ' Armed-Forces': 1, ' Craft-repair': 2, ' Exec-managerial': 3, ' Farming-fishing': 4, ' Handlers-cleaners': 5, ' Machine-op-inspct': 6, ' Other-service': 7, ' Priv-house-serv': 8, ' Prof-specialty': 9, ' Protective-serv': 10, ' Sales': 11, ' Tech-support': 12, ' Transport-moving': 13}, 'relationship': {' Husband': 0, ' Not-in-family': 1, ' Other-relative': 2, 

In [148]:
print(df.shape)
df.head(3)

(32561, 13)


,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,6,9,2,0,1,4,1,2174,0,40,38,0
1,50,5,9,1,3,0,4,1,0,0,13,38,0
2,38,3,11,0,5,1,4,1,0,0,40,38,0


In [157]:
from sklearn.model_selection import train_test_split
X = df.values[:, 0:12]
Y = df.values[:,12]



X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)


from sklearn.tree import DecisionTreeClassifier
dt_clf_gini = DecisionTreeClassifier(criterion = "gini", 
                                     random_state = 100,
                                     max_depth=5, 
                                     min_samples_leaf=5)


dt_clf_gini.fit(X_train, y_train)
y_pred_gini = dt_clf_gini.predict(X_test)


from sklearn.metrics import accuracy_score
print ("Desicion Tree using Gini Index\nAccuracy is : ", accuracy_score(y_test,y_pred_gini)*100 )

#creating and training a model
#serializing our model to a file called model.pkl
import pickle
pickle.dump(dt_clf_gini, open("model.pkl","wb"))

Desicion Tree using Gini Index
Accuracy is :  83.13031016480704


`Pickle is used for serializing and de-serializing a Python object structure.`